In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from bs4 import BeautifulSoup
import time

In [2]:
options = Options() 
chrome_path = r'\Users\User\Documents\chromedriver'

In [3]:
options.add_argument('--window-size=500,300')

In [4]:
driver = webdriver.Chrome(chrome_path, 
                              options=options)

driver.set_window_size(1400,1000)

In [19]:
driver.get('https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?HorseId=HK_2018_C336&Option=1')

In [20]:
import pandas as pd
page_source = driver.page_source
soup = BeautifulSoup(page_source)
tables = soup.findAll('table')
table = tables[0].prettify()
df = pd.read_html(table)[0]

In [21]:
name = df.loc[1][0]

In [22]:
df.drop(1, axis = 1, inplace = True)
df.drop([0,1,2,3,4], inplace = True)

In [23]:
y = df.set_index(0)
y = y.T

In [24]:
y['hname'] = name

In [25]:
y

,Country of Origin / Age,Colour / Sex,Import Type,Season Stakes*,Total Stakes*,No. of 1-2-3-Starts*,No. of starts in past 10 race meetings,Current Stable Location (Arrival Date),Trainer,Owner,Current Rating,Start of Season Rating,Sire,Dam,Dam's Sire,Same Sire,*Includes local and overseas records and earnings,hname
2,AUS / 3,Chestnut / Gelding,PPG,"$7,361,000","$9,116,600",5-0-2-7,0,Hong Kong (23/12/2018),J Moore,Andrea Tien & Arthur Cheng,119,89,Sebring,Pinocchio,Encosta de Lago,ALLOY STAR ASSIMILATE BIG LUCK CHAMP DIAMON...,*Includes local and overseas records and earnings,AETHERO (C336)


In [26]:
x = df.set_index(0)
x = x.T

In [27]:
x['hname'] = name

In [28]:
f = x

In [29]:
f.drop(2, inplace = True)

In [30]:
def frmt(df):
    global name
    name = df.loc[1][0]
    df.drop(1, axis = 1, inplace = True)
    df.drop([0,1,2,3,4], inplace = True)
    y = df.set_index(0)
    y = y.T
    y['hname'] = name
    global f
    f = pd.concat([f,y], ignore_index = True, sort = True)

In [31]:
page_source = driver.page_source
soup = BeautifulSoup(page_source)
tables = soup.find('table', attrs = {'class' : 'bigborder'})
table = tables.prettify()
df = pd.read_html(table, header = 0)[0]
df.dropna(how='all', inplace = True)
df.reset_index(drop = True, inplace = True)

In [32]:
plcmnt = df['Pla.'].values
tdrop = []
for i, val in enumerate(plcmnt):
    if val.endswith('Season'):
        tdrop.append(i)

In [33]:
tdrop

[0, 5]

In [34]:
df.drop(tdrop, inplace = True)

In [35]:
race_hist = df.drop([1,2,3,4,6,7,8])

In [36]:
g = race_hist

In [38]:
def prcs(df):
    plcmnt = df['Pla.'].values
    tdrop = []
    for i, val in enumerate(plcmnt):
        if val.endswith('Season'):
            tdrop.append(i)
    df.drop(tdrop, inplace = True)
    df['hname'] = name
    a = df
    global g
    g = pd.concat([g,a], ignore_index = True, sort = True)

In [39]:
def get_race_hist(df):
    try:
        driver.find_element_by_id('hf_allr_btn_r').click()
        page_source = driver.page_source
        soup = BeautifulSoup(page_source)
        tables = soup.find('table', attrs = {'class' : 'bigborder'})
        table = tables.prettify()
        df = pd.read_html(table, header = 0)[0]
        df.dropna(how='all', inplace = True)
        df.reset_index(drop = True, inplace = True)
        prcs(df)
    except:
        return 'nothin'

In [41]:
alpha = ['A','B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
row = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59','60','61','62','63','64','65','66','67','68','69','70','71','72','73','74','75','76','77','78','79','80']
column = ['1','2','3']
for letter in alpha:
    driver.get('https://racing.hkjc.com/racing/information/english/Horse/SelectHorsebyChar.aspx?ordertype=' + letter)
    try:
        for r in row:
            for c in column:
                time.sleep(1)
                driver.find_element_by_xpath('/html/body/div/p/table/tbody/tr[2]/td/table/tbody/tr[' + r + ']/td[' + c + ']/table/tbody/tr/td[1]/li/a').click()
                time.sleep(1)
                page_source = driver.page_source
                soup = BeautifulSoup(page_source)
                tables = soup.findAll('table')
                table = tables[0].prettify()
                df = pd.read_html(table)[0]
                frmt(df)
                time.sleep(1)
                get_race_hist(df)
                time.sleep(1)
                driver.get('https://racing.hkjc.com/racing/information/english/Horse/SelectHorsebyChar.aspx?ordertype=' + letter)
    except:
        continue

In [35]:
f.drop('*Includes local and overseas records and earnings', axis = 1, inplace = True)

In [36]:
f.drop('PP Pre-import races footage',axis = 1, inplace = True)

In [37]:
f.to_csv('horse_info.csv', index = False)

In [38]:
g.to_csv('race_hist.csv', index = False)

In [42]:
driver.quit()